In [1]:
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)
import os
os.chdir('./drive/My Drive/TESI/Autoencoder')

Mounted at /content/drive/


In [2]:
import pickle as pickle
import math

from src import BikeNYC, BikeNYC3d
import numpy as np

np.random.seed(1337)  # for reproducibility

# parameters
model_name = 'model3'
DATAPATH = '../data'
nb_epoch = 100  # number of epoch at training stage
# nb_epoch_cont = 150  # number of epoch at training (cont) stage
batch_size = 16  # batch size
T = 24  # number of time intervals in one day
CACHEDATA = True  # cache data or NOT

lr = 0.00015  # learning rate
len_closeness = 3  # length of closeness dependent sequence
len_period = 1  # length of peroid dependent sequence
len_trend = 1  # length of trend dependent sequence

nb_flow = 2  # there are two types of flows: new-flow and end-flow
# divide data into two subsets: Train & Test, of which the test set is the
# last 10 days
days_test = 10
len_test = T*days_test
len_val = 2*len_test

map_height, map_width = 16, 8  # grid size
# For NYC Bike data, there are 81 available grid-based areas, each of
# which includes at least ONE bike station. Therefore, we modify the final
# RMSE by multiplying the following factor (i.e., factor).
nb_area = 81
m_factor = math.sqrt(1. * map_height * map_width / nb_area)
# print('factor: ', m_factor)

cache_folder = 'Autoencoder/model3' if model_name == 'model3' else 'Autoencoder'
path_cache = os.path.join(DATAPATH, 'CACHE', cache_folder)  # cache path
path_result = 'RET'
path_model = 'MODEL'
if os.path.isdir(path_result) is False:
    os.mkdir(path_result)
if os.path.isdir(path_model) is False:
    os.mkdir(path_model)
if CACHEDATA and os.path.isdir(path_cache) is False:
    os.mkdir(path_cache)

In [3]:
from utils import cache, read_cache
import time

print("loading data...")
fname = os.path.join(path_cache, 'BikeNYC_C{}_P{}_T{}.h5'.format(
    len_closeness, len_period, len_trend))
if os.path.exists(fname) and CACHEDATA:
    X_train_all, Y_train_all, X_train, Y_train, \
    X_val, Y_val, X_test, Y_test, mmn, external_dim, \
    timestamp_train_all, timestamp_train, timestamp_val, timestamp_test = read_cache(
        fname, 'preprocessing_nyc.pkl')
    print("load %s successfully" % fname)
else:
    if (model_name == 'model3'):
        load_data = BikeNYC3d.load_data
    else:
        load_data = BikeNYC.load_data
    X_train_all, Y_train_all, X_train, Y_train, \
    X_val, Y_val, X_test, Y_test, mmn, external_dim, \
    timestamp_train_all, timestamp_train, timestamp_val, timestamp_test = load_data(
        T=T, nb_flow=nb_flow, len_closeness=len_closeness, len_period=len_period, len_trend=len_trend, len_test=len_test,
        len_val=len_val, preprocess_name='preprocessing_nyc.pkl', meta_data=True, datapath=DATAPATH)
    if CACHEDATA:
        cache(fname, X_train_all, Y_train_all, X_train, Y_train, X_val, Y_val, X_test, Y_test,
              external_dim, timestamp_train_all, timestamp_train, timestamp_val, timestamp_test)

print("\n days (test): ", [v[:8] for v in timestamp_test[0::T]])

loading data...
load ../data/CACHE/Autoencoder/model3/BikeNYC_C3_P1_T1.h5 successfully

 days (test):  [b'20140921', b'20140922', b'20140923', b'20140924', b'20140925', b'20140926', b'20140927', b'20140928', b'20140929', b'20140930']


/content/drive/My Drive/TESI/Autoencoder/utils.py:18: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  num = int(f['num'].value)
/content/drive/My Drive/TESI/Autoencoder/utils.py:21: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  X_train_all.append(f['X_train_all_%i' % i].value)
/content/drive/My Drive/TESI/Autoencoder/utils.py:22: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  X_train.append(f['X_train_%i' % i].value)
/content/drive/My Drive/TESI/Autoencoder/utils.py:23: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  X_val.append(f['X_val_%i' % i].value)
/content/drive/My Drive/TESI/Autoencoder/utils.py:24: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  X_test.append(f['X_test_%i' % i].value)
/content/drive/My Drive/TESI/Autoencoder/utils.py:25: H5pyDeprecationWarning: dataset.value has been dep

In [4]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from utils import build_model

model = build_model(
    len_closeness, len_period, len_trend, nb_flow, map_height, map_width,
    model = model_name,
    external_dim=external_dim, lr=lr,
    encoder_blocks=2,
    filters=[32,64,16],
    save_model_pic=f'BikeNYC_{model_name}'
)
# model.summary()
hyperparams_name = '{}.BikeNYC.c{}.p{}.t{}.lr{}'.format(
    model_name, len_closeness, len_period, len_trend, lr)
fname_param = os.path.join('MODEL', '{}.best.h5'.format(hyperparams_name))

early_stopping = EarlyStopping(monitor='val_rmse', patience=5, mode='min')
model_checkpoint = ModelCheckpoint(
    fname_param, monitor='val_rmse', verbose=0, save_best_only=True, mode='min')

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file=f'BikeNYC_{model_name}.png', show_shapes=True, show_layer_names=False)

In [5]:
# train model
print("training model...")
ts = time.time()
history = model.fit(X_train, Y_train,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    validation_data=(X_val,Y_val),
                    # callbacks=[early_stopping, model_checkpoint],
                    callbacks=[model_checkpoint],
                    verbose=2)
model.save_weights(os.path.join(
    'MODEL', '{}.h5'.format(hyperparams_name)), overwrite=True)
pickle.dump((history.history), open(os.path.join(
    path_result, '{}.history.pkl'.format(hyperparams_name)), 'wb'))
print("\nelapsed time (training): %.3f seconds\n" % (time.time() - ts))

training model...
Epoch 1/100
234/234 - 18s - loss: 0.4186 - rmse: 0.6140 - val_loss: 0.4576 - val_rmse: 0.6764
Epoch 2/100
234/234 - 16s - loss: 0.1220 - rmse: 0.3475 - val_loss: 0.1215 - val_rmse: 0.3485
Epoch 3/100
234/234 - 16s - loss: 0.0522 - rmse: 0.2132 - val_loss: 0.0099 - val_rmse: 0.0984
Epoch 4/100
234/234 - 16s - loss: 0.0069 - rmse: 0.0820 - val_loss: 0.0064 - val_rmse: 0.0786
Epoch 5/100
234/234 - 16s - loss: 0.0051 - rmse: 0.0706 - val_loss: 0.0051 - val_rmse: 0.0703
Epoch 6/100
234/234 - 16s - loss: 0.0043 - rmse: 0.0646 - val_loss: 0.0050 - val_rmse: 0.0690
Epoch 7/100
234/234 - 17s - loss: 0.0038 - rmse: 0.0608 - val_loss: 0.0040 - val_rmse: 0.0618
Epoch 8/100
234/234 - 16s - loss: 0.0034 - rmse: 0.0578 - val_loss: 0.0037 - val_rmse: 0.0597
Epoch 9/100
234/234 - 16s - loss: 0.0031 - rmse: 0.0550 - val_loss: 0.0034 - val_rmse: 0.0568
Epoch 10/100
234/234 - 15s - loss: 0.0029 - rmse: 0.0530 - val_loss: 0.0034 - val_rmse: 0.0573
Epoch 11/100
234/234 - 16s - loss: 0.0027

In [6]:
# evaluate
model.load_weights(fname_param)
# model.load_weights('MODEL/BikeNYC.c3.p1.t1.lr0.00015.best.h5')
score = model.evaluate(
    X_test, Y_test, batch_size=Y_test.shape[0], verbose=0)
print('Test score: %.6f rmse (norm): %.6f rmse (real): %.6f' %
        (score[0], score[1], score[1] * (mmn._max - mmn._min) / 2.))

Test score: 0.001425 rmse (norm): 0.037753 rmse (real): 5.040021
